<a href="https://colab.research.google.com/github/juliaeduarda-rg/Dados-Study/blob/main/ENG_dados_Sicrona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [34]:
credencial = service_account.Credentials.from_service_account_file(
    '/content/juliaproject-sirius-d17644e61f58.json'
)

In [35]:
client = bigquery.Client(credentials= credencial, project= credencial.project_id)

In [36]:
df = client.query('''SELECT *
FROM `basedosdados.br_ibge_populacao.municipio` 
where ANO = 2021 and populacao > 100000
LIMIT 1000''').to_dataframe()

In [37]:
df.head()

,ano,sigla_uf,id_municipio,populacao
0,2021,RO,1100023,111148
1,2021,RO,1100122,131026
2,2021,RO,1100205,548952
3,2021,RO,1100304,104517
4,2021,AC,1200401,419452


In [38]:
import pandas as pd
import requests as rq
import json as js
import numpy as np

In [39]:
chave = 'XXXXX'

In [40]:
chave_dict = {'chave-api-dados': chave}

In [41]:
df_lista = list(df['id_municipio'])
id_municipio = df_lista

In [42]:
lista_result= []
cont = 0
for id in df_lista:
  url = f'https://api.portaldatransparencia.gov.br/api-de-dados/bolsa-familia-por-municipio?codigoIbge={id_municipio[cont]}&mesAno=202111&pagina=1'
  cont +=1
  result = rq.get(url, verify=True, headers=chave_dict).json()
  lista_result+= result


In [52]:
df_result = pd.json_normalize(lista_result)

In [53]:
df_result

,id,dataReferencia,valor,quantidadeBeneficiados,municipio.codigoIBGE,municipio.nomeIBGE,municipio.codigoRegiao,municipio.nomeRegiao,municipio.pais,municipio.uf.sigla,municipio.uf.nome,tipo.id,tipo.descricao,tipo.descricaoDetalhada
0,201902817,2021-11-01,841397.0,4403,1100023,ARIQUEMES,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
1,207702394,2021-11-01,475821.0,2505,1100122,JI-PARANÁ,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
2,187901825,2021-11-01,5056739.0,25485,1100205,PORTO VELHO,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
3,212903389,2021-11-01,517514.0,2701,1100304,VILHENA,1,NORTE,BRASIL,RO,RONDÔNIA,1,Bolsa Família,Bolsa Família
4,210202149,2021-11-01,6976427.0,25773,1200401,RIO BRANCO,1,NORTE,BRASIL,AC,ACRE,1,Bolsa Família,Bolsa Família
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,202703801,2021-11-01,1337019.0,6626,5218805,RIO VERDE,5,CENTRO-OESTE,BRASIL,GO,GOIÁS,1,Bolsa Família,Bolsa Família
322,210203600,2021-11-01,1019396.0,5180,5220454,SENADOR CANEDO,5,CENTRO-OESTE,BRASIL,GO,GOIÁS,1,Bolsa Família,Bolsa Família
323,211602921,2021-11-01,1217517.0,5986,5221403,TRINDADE,5,CENTRO-OESTE,BRASIL,GO,GOIÁS,1,Bolsa Família,Bolsa Família
324,204302480,2021-11-01,1285512.0,7131,5221858,VALPARAÍSO DE GOIÁS,5,CENTRO-OESTE,BRASIL,GO,GOIÁS,1,Bolsa Família,Bolsa Família


porque precisa de 2 [[]] nessa parte e um só não funciona?

In [59]:
df_result.rename(columns={'municipio.codigoIBGE': 'id_municipio'}, inplace = True)

In [60]:
df_final_BF = df_result[['id_municipio', 'quantidadeBeneficiados', 'valor', 'dataReferencia']]

In [61]:
df_final = df.merge(df_final_BF, how = 'inner', on= 'id_municipio')

In [62]:
df_final

,ano,sigla_uf,id_municipio,populacao,quantidadeBeneficiados,valor,dataReferencia
0,2021,RO,1100023,111148,4403,841397.0,2021-11-01
1,2021,RO,1100122,131026,2505,475821.0,2021-11-01
2,2021,RO,1100205,548952,25485,5056739.0,2021-11-01
3,2021,RO,1100304,104517,2701,517514.0,2021-11-01
4,2021,AC,1200401,419452,25773,6976427.0,2021-11-01
...,...,...,...,...,...,...,...
321,2021,GO,5218805,247259,6626,1337019.0,2021-11-01
322,2021,GO,5220454,121447,5180,1019396.0,2021-11-01
323,2021,GO,5221403,132006,5986,1217517.0,2021-11-01
324,2021,GO,5221858,175720,7131,1285512.0,2021-11-01
